In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        (os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# =====================================
# A → Z SINGLE CELL CODE
# Brain Tumor MRI (4 Classes)
# Multi-Scale CNN + BiGRU + Attention
# =====================================

import os
import cv2
import numpy as np
import tensorflow as tf

from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import (
    Input, Conv2D, Concatenate, Reshape,
    Bidirectional, GRU, Dense, Dropout,
    BatchNormalization, Layer
)
from tensorflow.keras.models import Model

# ===============================
# CONFIGURATION
# ===============================
DATASET_PATH = "/kaggle/input/brain-tumor-mri-dataset/Training"
CLASS_NAMES = ["glioma", "meningioma", "notumor", "pituitary"]
NUM_CLASSES = 4
IMG_SIZE = 128

# ===============================
# LOAD DATASET → NUMPY
# ===============================
X, y = [], []

for idx, class_name in enumerate(CLASS_NAMES):
    class_path = os.path.join(DATASET_PATH, class_name)
    for img_name in os.listdir(class_path):
        img_path = os.path.join(class_path, img_name)

        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))

        X.append(img)
        y.append(idx)

X = np.array(X, dtype=np.float32) / 255.0
y = to_categorical(np.array(y), NUM_CLASSES)

print("Dataset Loaded")
print("X shape:", X.shape)   # (N, 128, 128, 3)
print("y shape:", y.shape)

# ===============================
# TRAIN / VALIDATION SPLIT
# ===============================
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# ===============================
# ATTENTION POOLING
# ===============================
class AttentionPooling(Layer):
    def build(self, input_shape):
        self.W = self.add_weight(
            shape=(input_shape[-1], 1),
            initializer="normal",
            trainable=True
        )

    def call(self, x):
        score = tf.nn.softmax(tf.matmul(x, self.W), axis=1)
        return tf.reduce_sum(x * score, axis=1)

# ===============================
# MODEL
# ===============================
inputs = Input(shape=(IMG_SIZE, IMG_SIZE, 3))

# Multi-scale CNN
c1 = Conv2D(32, 3, padding="same", activation="relu")(inputs)
c2 = Conv2D(32, 5, padding="same", activation="relu")(inputs)
c3 = Conv2D(32, 7, padding="same", activation="relu")(inputs)

cnn = Concatenate()([c1, c2, c3])
cnn = BatchNormalization()(cnn)

# Convert CNN feature maps → sequence
cnn = Reshape((cnn.shape[1] * cnn.shape[2], cnn.shape[3]))(cnn)

# BiGRU
x = Bidirectional(GRU(128, return_sequences=True))(cnn)
x = Dropout(0.3)(x)

# Attention
x = AttentionPooling()(x)

# Classifier
x = Dense(128, activation="relu")(x)
x = Dropout(0.4)(x)
outputs = Dense(NUM_CLASSES, activation="softmax")(x)

model = Model(inputs, outputs)
model.summary()

# ===============================
# COMPILE & TRAIN
# ===============================
model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=20,
    batch_size=16,
    verbose=1
)

# ===============================
# SAVE MODEL
# ===============================
model.save("BrainTumor_MultiScaleCNN_BiGRU_Attention.h5")
print("Model saved successfully")


Dataset Loaded
X shape: (5712, 128, 128, 3)
y shape: (5712, 4)


I0000 00:00:1766518771.240806      47 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 128, 128,  │        896 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 128, 128,  │      2,432 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 128, 128,  │      4,736 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 128, 128,  │          0 │ conv2d[0][0],     │
│ (Concatenate)       │ 96)               │            │ conv2d_1[0][0],   │
│                     │                   │            │ conv2d_2[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 128, 128,  │        384 │ concatenate[0][0] │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape (Reshape)   │ (None, 16384, 96) │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional       │ (None, 16384,     │    173,568 │ reshape[0][0]     │
│ (Bidirectional)     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 16384,     │          0 │ bidirectional[0]… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attention_pooling   │ (None, 256)       │        256 │ dropout[0][0]     │
│ (AttentionPooling)  │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 128)       │     32,896 │ attention_poolin… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 128)       │          0 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 4)         │        516 │ dropout_1[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 215,684 (842.52 KB)

 Trainable params: 215,492 (841.77 KB)

 Non-trainable params: 192 (768.00 B)

Epoch 1/20


I0000 00:00:1766518778.448660     112 cuda_dnn.cc:529] Loaded cuDNN version 90300


 80/286 ━━━━━━━━━━━━━━━━━━━━ 3:09 919ms/step - accuracy: 0.4267 - loss: 1.2237